<a href="https://colab.research.google.com/github/Joagai23/SistemasInteligentesPokerBot/blob/master/Poker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [198]:
!pip install PyPokerEngine
!pip install scikit-fuzzy

# Fuzzy Logic

In [0]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

In [0]:
#Fuzzy1
#Antecedentes: ronda, porcentage, dinero
#Consecuencia:accion
#acciones->abandonar, seguir, apostar
rounds = ctrl.Antecedent(np.arange(0, 4, 1), 'Round')
percentage = ctrl.Antecedent(np.arange(0, 1.01, 0.1), 'Percentage') 
money = ctrl.Antecedent(np.arange(0, 101, 1), 'Money')

act = ctrl.Consequent(np.arange(0, 1.01, 0.1), 'Action')

In [0]:
rounds['Round0'] = fuzz.trimf(rounds.universe, [0, 0, 1])
rounds['Round1'] = fuzz.trimf(rounds.universe, [1, 1, 2])
rounds['Round2'] = fuzz.trimf(rounds.universe, [2, 2, 3])
rounds['Round3'] = fuzz.trimf(rounds.universe, [3, 3, 4])

In [0]:
percentage['Very Low'] = fuzz.trimf(percentage.universe, [0, 0, 0.25])
percentage['Low'] = fuzz.trimf(percentage.universe, [0, 0.25, 0.5])
percentage['Medium'] = fuzz.trimf(percentage.universe, [0.25, 0.5, 0.75])
percentage['High'] = fuzz.trimf(percentage.universe, [0.5, 0.75, 1])
percentage['Very High'] = fuzz.trimf(percentage.universe, [0.75, 1, 1])

In [0]:
money['Low'] = fuzz.trimf(money.universe, [0, 0, 30])
money['Medium'] = fuzz.trimf(money.universe, [25, 50, 75])
money['High'] = fuzz.trimf(money.universe, [70, 100, 100])

In [0]:
act['Fold'] = fuzz.trimf(act.universe, [0, 0, 0.36])
act['Call'] = fuzz.trimf(act.universe, [0.36, 0.653, 0.91])
act['Raise'] = fuzz.trimf(act.universe, [0.91, 1, 1])

In [0]:
#RULES para ver si abandonar, seguir o apostar
rule1  = ctrl.Rule(percentage['Very Low'],                                                                                                         act['Fold'])
rule2  = ctrl.Rule(percentage['Very High'],                                                                                                        act['Raise'])
rule3  = ctrl.Rule(rounds['Round0']    &   (percentage['High']   | percentage['Medium'] | percentage['Low']),                                      act['Call'])
rule4  = ctrl.Rule(rounds['Round1']    &    percentage['Low']                                                  &  money['Low'],                    act['Fold'])
rule5  = ctrl.Rule(rounds['Round1']    &    percentage['Low']                                                  & (money['Medium'] | money['Low']), act['Call'])
rule6  = ctrl.Rule(rounds['Round1']    &   (percentage['Medium'] | percentage['High']),                                                            act['Call'])
rule7  = ctrl.Rule(rounds['Round2']    &   (percentage['Low']    | percentage['Medium'])                       & (money['Medium'] | money['Low']), act['Fold'])
rule8  = ctrl.Rule(rounds['Round2']    &   (percentage['Low']    | percentage['Medium'])                       &  money['High'],                   act['Call'])
rule9  = ctrl.Rule(rounds['Round2']    &    percentage['High'],                                                                                    act['Call'])
rule10 = ctrl.Rule(rounds['Round3']    &   (percentage['Low']    | percentage['Medium']),                                                          act['Fold'])
rule11 = ctrl.Rule(rounds['Round3']    &   percentage['High'],                                                                                     act['Call'])

In [0]:
action_ctrl = ctrl.ControlSystem([rule1,  rule2,  rule3,  rule4,  rule5,  rule6,  rule7,  rule8,  rule9, rule10, rule11])

In [0]:
action = ctrl.ControlSystemSimulation(action_ctrl)

In [0]:
def getAction(roundNum, percentageVal, moneyVal):
    
    # Pass inputs to the ControlSystem using Antecedent labels with Pythonic API
    # Note: if you like passing many inputs all at once, use .inputs(dict_of_data)
    action.input['Round'] = roundNum
    action.input['Percentage'] = percentageVal
    action.input['Money'] = moneyVal


    # Crunch the numbers
    action.compute()

    string = ""

    if percentageVal < 0.36:
      string = "Fold"
    elif percentageVal >= 0.36 and percentageVal < 0.91:
      string = "Call"
    elif percentageVal >= 0.91 and percentageVal <= 1:
      string = "Raise"
    
    print("Action:", string)
    
    return string

In [0]:
#Fuzzy2
#Antecedentes: ronda, porcentage, dinero
#Consecuencia:porcentage de tu dinero que debes apostar
#correlacion con la anterior, se ejecuta siempre
rounds2 = ctrl.Antecedent(np.arange(0, 4, 1), 'Round')
percentage2 = ctrl.Antecedent(np.arange(0, 1.01, 0.1), 'Percentage') 
money2 = ctrl.Antecedent(np.arange(0, 101, 1), 'Money')

moneyPerc = ctrl.Consequent(np.arange(0, 1.01, 0.1), 'Percentageofyourmoney')

In [0]:
rounds2['Round0'] = fuzz.trimf(rounds2.universe, [0, 0, 1])
rounds2['Round1'] = fuzz.trimf(rounds2.universe, [1, 1, 2])
rounds2['Round2'] = fuzz.trimf(rounds2.universe, [2, 2, 3])
rounds2['Round3'] = fuzz.trimf(rounds2.universe, [3, 3, 4])
#rounds2.view()

In [0]:
percentage2['Very Low'] = fuzz.trimf(percentage2.universe, [0, 0, 0.25])
percentage2['Low'] = fuzz.trimf(percentage2.universe, [0, 0.25, 0.5])
percentage2['Medium'] = fuzz.trimf(percentage2.universe, [0.25, 0.5, 0.75])
percentage2['High'] = fuzz.trimf(percentage2.universe, [0.5, 0.75, 1])
percentage2['Very High'] = fuzz.trimf(percentage2.universe, [0.75, 1, 1])
#percentage2.view()

In [0]:
money2['Low'] = fuzz.trimf(money2.universe, [0, 0, 30])
money2['Medium'] = fuzz.trimf(money2.universe, [25, 50, 75])
money2['High'] = fuzz.trimf(money2.universe, [70, 100, 100])
#money2.view()

In [0]:
#porcentage de tu dinero que vas a apostar
moneyPerc['Very Low'] = fuzz.trimf(moneyPerc.universe, [0, 0, 0.1])
moneyPerc['Low'] = fuzz.trimf(moneyPerc.universe, [0.05, 0.2, 0.35])
moneyPerc['Medium'] = fuzz.trimf(moneyPerc.universe, [0.25, 0.56, 0.75])
moneyPerc['High'] = fuzz.trimf(moneyPerc.universe, [0.65, 0.8, 0.95])
moneyPerc['Very High'] = fuzz.trimf(moneyPerc.universe, [0.9, 1, 1])
#moneyPerc.view()

In [0]:
rule21  = ctrl.Rule(percentage2['Very Low'],                                                                                                         moneyPerc['Very Low']) #FOLD
rule22  = ctrl.Rule(percentage2['Very High'],                                                                                                        moneyPerc['Very High']) #RAISE
rule23  = ctrl.Rule(rounds2['Round0']    &   (percentage2['Medium'] | percentage2['Low']),                                                              moneyPerc['Very Low']) #CALL
rule24  = ctrl.Rule(rounds2['Round0']    &   percentage2['High'],                                                                                        moneyPerc['Low']) #CALL
rule25  = ctrl.Rule(rounds2['Round1']    &    percentage2['Low']                                                  &  money2['Low'],                    moneyPerc['Very Low']) #FOLD
rule26  = ctrl.Rule(rounds2['Round1']    &    percentage2['Low']                                                  & (money2['Medium'] | money2['Low']), moneyPerc['Low']) #CALL
rule27  = ctrl.Rule(rounds2['Round1']    &   (percentage2['Medium'] | percentage2['High']),                                                            moneyPerc['Medium']) #CALL
rule28  = ctrl.Rule(rounds2['Round2']    &   (percentage2['Low']    | percentage2['Medium'])                       & (money2['Medium'] | money2['Low']), moneyPerc['Very Low']) #FOLD
rule29  = ctrl.Rule(rounds2['Round2']    &   (percentage2['Low']    | percentage2['Medium'])                       &  money2['High'],                   moneyPerc['Medium']) #CALL
rule30  = ctrl.Rule(rounds2['Round2']    &    percentage2['High'],                                                                                    moneyPerc['High']) #CALL OR RAISE
rule31 = ctrl.Rule(rounds2['Round3']    &   (percentage2['Low']    | percentage2['Medium']),                                                          moneyPerc['Very Low']) #FOLD
rule32 = ctrl.Rule(rounds2['Round3']    &   percentage2['High'],                                                                                     moneyPerc['High']) #CALL OR RAISE

In [0]:
action_ctrl2 = ctrl.ControlSystem([rule21,  rule22,  rule23,  rule24,  rule25,  rule26,  rule27,  rule28,  rule29, rule30, rule31, rule32])

In [0]:
action2 = ctrl.ControlSystemSimulation(action_ctrl2)

In [0]:
import math

def getMoneyPercentage(roundNum, percentageVal, moneyVal):

    # Pass inputs to the ControlSystem using Antecedent labels with Pythonic API
    # Note: if you like passing many inputs all at once, use .inputs(dict_of_data)
    action2.input['Round'] = roundNum
    action2.input['Percentage'] = percentageVal
    action2.input['Money'] = moneyVal

    # Crunch the numbers
    action2.compute()

    dollars = action2.output['Percentageofyourmoney']
    
    return dollars

# **Poker Engine**

In [0]:
import pypokerengine

## **Players**

### Fish Player

In [0]:
from pypokerengine.players import BasePokerPlayer

class FishPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"

    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

### Console Player

In [0]:
import pypokerengine.utils.visualize_utils as U

class ConsolePlayer(BasePokerPlayer):

    def declare_action(self, valid_actions, hole_card, round_state):
        print(U.visualize_declare_action(valid_actions, hole_card, round_state, self.uuid))
        action, amount = self._receive_action_from_console(valid_actions)
        return action, amount

    def receive_game_start_message(self, game_info):
        print(U.visualize_game_start(game_info, self.uuid))
        self._wait_until_input()

    def receive_round_start_message(self, round_count, hole_card, seats):
        print(U.visualize_round_start(round_count, hole_card, seats, self.uuid))
        self._wait_until_input()

    def receive_street_start_message(self, street, round_state):
        print(U.visualize_street_start(street, round_state, self.uuid))
        self._wait_until_input()

    def receive_game_update_message(self, new_action, round_state):
        print(U.visualize_game_update(new_action, round_state, self.uuid))
        self._wait_until_input()

    def receive_round_result_message(self, winners, hand_info, round_state):
        print(U.visualize_round_result(winners, hand_info, round_state, self.uuid))
        self._wait_until_input()

    def _wait_until_input(self):
        input("Enter some key to continue ...")

    # FIXME: This code would be crash if receives invalid input.
    #        So you should add error handling properly.
    def _receive_action_from_console(self, valid_actions):
        action = input("Enter action to declare >> ")
        if action == 'fold': amount = 0
        if action == 'call':  amount = valid_actions[1]['amount']
        if action == 'raise':  amount = int(input("Enter raise amount >> "))
        return action, amount

### Fuzzy Player

In [0]:
NB_SIMULATION = 2000
NB_PLAYER = 2

In [0]:
import pypokerengine.utils.visualize_utils as U
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate

class FuzzyPlayer(BasePokerPlayer):

    def declare_action(self, valid_actions, hole_card, round_state):
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(
                nb_simulation=NB_SIMULATION,
                nb_player=NB_PLAYER,
                hole_card=gen_cards(hole_card),
                community_card=gen_cards(community_card)
                )
        ronda = round_state["street"]
        numRound = 0

        if ronda == 'preflop': numRound = 0
        if ronda == 'flop':  numRound = 1
        if ronda == 'turn':  numRound = 2
        if ronda == 'river':  numRound = 3

        M = round_state["seats"][0]["stack"] #player's money
        action = getAction(roundNum=numRound, percentageVal=win_rate, moneyVal=M) #first fuzzy
        action = action.lower()
        
        bet = getMoneyPercentage(roundNum=numRound, percentageVal=win_rate, moneyVal=M) #second fuzzy (money)

        if action == 'fold': amount = 0
        if action == 'call':  amount = valid_actions[1]['amount']
        if action == 'raise':  amount = int(bet * M)
      
        return action, amount

    def receive_game_start_message(self, game_info):
        print(U.visualize_game_start(game_info, self.uuid))

    def receive_round_start_message(self, round_count, hole_card, seats):
        print(U.visualize_round_start(round_count, hole_card, seats, self.uuid))

    def receive_street_start_message(self, street, round_state):
        print(U.visualize_street_start(street, round_state, self.uuid))

    def receive_game_update_message(self, new_action, round_state):
        print(U.visualize_game_update(new_action, round_state, self.uuid))
    def receive_round_result_message(self, winners, hand_info, round_state):
        print(U.visualize_round_result(winners, hand_info, round_state, self.uuid))

## **Game Config**

In [230]:
from pypokerengine.api.game import setup_config, start_poker

config = setup_config(max_round=3, initial_stack=100, small_blind_amount=5)
config.register_player(name="fuzzy_player", algorithm=FuzzyPlayer())
config.register_player(name="fish_player", algorithm=FishPlayer())
#config.register_player(name="human_player", algorithm=ConsolePlayer())
numPlayers=fishPlayers
game_result = start_poker(config, verbose=0)

-- Game start (UUID = sisdpcgxtsovibgystosmm) --
-- rule --
  - 2 players game
  - 3 round
  - start stack = 100
  -        ante = 0
  - small blind = 5
-- Round 1 start (UUID = sisdpcgxtsovibgystosmm) --
-- hole card --
  - ['D9', 'CK']
-- players information --
  - 0 : fuzzy_player (sisdpcgxtsovibgystosmm) => state : participating, stack : 90
  - 1 : fish_player (nzfmicdekzbyknogiolgez) => state : participating, stack : 95
-- New street start (UUID = sisdpcgxtsovibgystosmm) --
-- street --
  - preflop
-- Game update (UUID = sisdpcgxtsovibgystosmm) --
-- new action --
  - fish_player (nzfmicdekzbyknogiolgez) declared call: 10
-- round state --
  - dealer btn : fuzzy_player
  - street : preflop
  - community card : []
  - pot : main = 20, side = []
  - players information
    - 0 : fuzzy_player (sisdpcgxtsovibgystosmm) => state : participating, stack : 90 <= BB
    - 1 : fish_player (nzfmicdekzbyknogiolgez) => state : participating, stack : 90 <= SB, CURRENT
  - action histories
    - 